In [2]:
import pandas as pd 
from pandas import ExcelWriter
from pandas import ExcelFile 
import numpy as np 

from IPython.display import display 

data = pd.read_excel('Clean_PA_Data_trunk.xlsx', sheetname ='Sheet1')

print (data.columns)

display(data.head(n=5))

y = data['Label']

erase = ['Label', 'Usage Period 1']

X = data.drop(data[erase], axis = 1)



Index(['Label', 'Service Same as Billing ', 'Billing State', 'Load Factor',
       'Current PLC', 'Usage Period 1', 'Current NSPL', 'POLR Indicator',
       'Interval Meter Indicator', 'Future PLC',
       'Net Metering/ Renewables rider', 'Tax Status Indicator', 'Energyadj',
       'capacity', 'trans', 'margin per MWh', 'Annual Margin'],
      dtype='object')


,Label,Service Same as Billing,Billing State,Load Factor,Current PLC,Usage Period 1,Current NSPL,POLR Indicator,Interval Meter Indicator,Future PLC,Net Metering/ Renewables rider,Tax Status Indicator,Energyadj,capacity,trans,margin per MWh,Annual Margin
0,1,1,1,0.0,2.33,1553.0,2.76,0,1,2.68,0,0,40.406039,12.012827,4.081764,7.655179,109.974298
1,1,1,1,0.0,2.33,1711.0,2.76,0,1,2.68,0,0,39.948752,9.112216,3.096183,11.998656,227.242542
2,1,1,1,0.0,4.23,3029.0,4.96,0,1,4.81,0,0,41.367063,13.719505,4.659307,4.409932,99.739443
3,1,1,1,0.0,4.23,3357.0,2.76,0,1,2.68,0,0,43.788157,10.415148,3.138440,6.814062,127.313939
4,1,0,1,0.0,2.33,1784.0,2.76,0,1,2.68,0,0,41.100287,11.590078,3.938121,7.527322,112.081823


In [3]:
#fit unnormalized data 

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =.1, random_state = 0)

print(X_train.shape[0])

print(X_test.shape[0])


89
10


In [4]:
from sklearn.metrics import fbeta_score, make_scorer, accuracy_score, recall_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.grid_search import GridSearchCV

import Visuals as vs

beta =.6

parameters = {'n_estimators' : [1,2,3,6,10,100]}

model = AdaBoostClassifier(random_state = 0)

scorer = make_scorer(fbeta_score, beta = beta)

grid_obj = GridSearchCV(model, parameters, scoring=scorer)

grid_fit = grid_obj.fit(X_train, y_train)

best_model = grid_fit.best_estimator_

# don't test model on the test data 
#predictions_train = model.predict(X_train)

#predictions_test = model.predict(X_test)

best_predictions_test = best_model.predict(X_test)

importances = best_model.feature_importances_ 

#ftraining_score = fbeta_score(y_train, predictions_train, beta)
#ftest_score = fbeta_score(y_test, predictions_test, beta)

fbest_test_score = fbeta_score(y_test, best_predictions_test, beta)

acc_test_score = accuracy_score(y_test, best_predictions_test, beta) 

rec_test_score = recall_score(y_test, best_predictions_test) 

print (fbest_test_score, acc_test_score, rec_test_score)

#vs.feature_plot(importances, X_train, y_train)

print(X.columns)
print(importances)

C:\Users\dm12505\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\dm12505\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.441558441558 0.7 0.333333333333
Index(['Service Same as Billing ', 'Billing State', 'Load Factor',
       'Current PLC', 'Current NSPL', 'POLR Indicator',
       'Interval Meter Indicator', 'Future PLC',
       'Net Metering/ Renewables rider', 'Tax Status Indicator', 'Energyadj',
       'capacity', 'trans', 'margin per MWh', 'Annual Margin'],
      dtype='object')
[ 0.16666667  0.          0.          0.          0.          0.16666667
  0.          0.16666667  0.          0.          0.16666667  0.
  0.16666667  0.          0.16666667]
